In [1]:
dir = 'KG82sd.d'
dir = 'l22x6.d'

def filename(basename):
    return dir + '/' + basename + '.csv'

def Warn(msg):
    print('!!!!! Warning: {}'.format(msg))

In [2]:
import pandas as pd
import math

In [3]:
class Node(object):
    
    def __init__(self,id,x,y):
        self.id = id
        self.x = x
        self.y = y
        self.sumFX = 0.
        self.sumFY = 0.
        self.sumMZ = 0.

In [4]:
table = pd.read_csv(filename('nodes'))
NODES = {}
for i,n in table.iterrows():
    if n.ID in NODES:
        Warn("Node '{}' is multiply defined.".format(n.ID))
    NODES[n.ID] = Node(n.ID,float(n.X),float(n.Y))

In [5]:
class Member(object):
    
    def __init__(self,id,nodej,nodek):
        self.id = id
        self.nodej = nodej
        self.nodek = nodek
        
        dx = nodek.x - nodej.x
        dy = nodek.y - nodej.y
        self.L = L = math.sqrt(dx*dx + dy*dy)
        self.cosx = dx/L
        self.cosy = dy/L

In [6]:
table = pd.read_csv(filename('members'))
MEMBERS = {}
for i,m in table.iterrows():
    if m.ID in MEMBERS:
        Warn("Member '{}' is multiply defined.".format(m.ID))
    MEMBERS[m.ID] = Member(m.ID,NODES[m.NODEJ],NODES[m.NODEK])

In [7]:
table = pd.read_csv(filename('node_loads'))
for i,p in table.iterrows():
    dirn = p.DIRN.upper()
    if dirn in ['FX','FY','MZ']:
        n = NODES[p.ID]
        a = 'sum'+dirn
        setattr(n,a,getattr(n,a,0.)+float(p.F))
    else:
        Warn("Direction '{}' invalid for node '{}'.".format(dirn,p.ID))

In [8]:
table = pd.read_csv(filename('reactions'))
for i,r in table.iterrows():
    n = NODES[r.ID]
    n.sumFX += 0. if pd.isnull(r.FX) else float(r.FX)
    n.sumFY += 0. if pd.isnull(r.FY) else float(r.FY)
    n.sumMZ += 0. if pd.isnull(r.MZ) else float(r.MZ)

In [9]:
mtable = pd.read_csv(filename('mefs'))
for i,row in mtable.iterrows():
    m = MEMBERS[row.ID]
    n = m.nodej
    n.sumFX -= row.FXJ*m.cosx - row.FYJ*m.cosy
    n.sumFY -= row.FXJ*m.cosy + row.FYJ*m.cosx
    n.sumMZ -= row.MZJ
    n = m.nodek
    n.sumFX -= row.FXK*m.cosx - row.FYK*m.cosy
    n.sumFY -= row.FXK*m.cosy + row.FYK*m.cosx
    n.sumMZ -= row.MZK
mtable

,ID,FXJ,FYJ,MZJ,FXK,FYK,MZK
0,CA0A1,2244909.63470,588097.363747,2.105832e+09,-2244909.63470,-588097.363747,1.716801e+09
1,CB0B1,14519890.36530,636651.974250,2.211842e+09,-14519890.36530,-636651.974250,1.926396e+09
2,CC0C1,5419941.95971,581603.210063,2.095291e+09,-5419941.95971,-581603.210063,1.685130e+09
3,CD0D1,11345267.39970,175620.334257,-4.470348e-08,-11345267.39970,-175620.334257,1.141532e+09
4,CE0E1,15682790.64060,138388.366032,1.192093e-07,-15682790.64060,-138388.366032,8.995244e+08
5,CF0F1,4984721.95980,600484.824260,2.136854e+09,-4984721.95980,-600484.824260,1.766297e+09
6,CG0G1,11780078.04020,646153.927400,2.233926e+09,-11780078.04020,-646153.927400,1.966075e+09
7,CA1A2,2545848.76945,355440.190027,9.582551e+08,-2545848.76945,-355440.190027,9.966659e+08
8,CB1B2,13450151.23060,472873.631957,1.292609e+09,-13450151.23060,-472873.631957,1.308195e+09
9,CC1C2,5402913.82230,348568.583511,9.119716e+08,-5402913.82230,-348568.583511,1.005156e+09


In [10]:
sums = pd.DataFrame([(n.id,n.sumFX,n.sumFY,n.sumMZ) for n in NODES.values()],
                    columns=['ID','sumFX','sumFY','sumMZ']).set_index(['ID'])
sums

,sumFX,sumFY,sumMZ
ID,,,
D14,-3.999958e-07,5.300324e-06,-4.999995e-03
A1,-2.000161e-07,-6.000220e-06,2.999783e-03
C18,-1.000008e-06,6.999908e-06,0.000000e+00
B3,-5.999173e-07,4.999922e-06,-4.768372e-07
G10,6.000337e-07,-9.991927e-07,2.000093e-03
B11,9.999640e-07,-2.000423e-06,3.999949e-03
E2,3.000714e-07,-1.399871e-05,0.000000e+00
E1,0.000000e+00,-5.300099e-05,1.999855e-03
B7,8.999486e-07,-6.001093e-06,-3.999949e-03


In [11]:
sums.abs().max()

sumFX    0.000002
sumFY    0.000059
sumMZ    0.010000
dtype: float64

In [12]:
from numpy.linalg import norm

In [13]:
norm(sums.sumFX,2)/min(norm(mtable.FXJ,2),norm(mtable.FXK,2))

9.6453293201111507e-14

In [14]:
norm(sums.sumFY,2)/min(norm(mtable.FYJ,2),norm(mtable.FYK,2))

3.2521548506351771e-11

In [15]:
norm(sums.sumMZ,2)/min(norm(mtable.MZJ,2),norm(mtable.MZK,2))

2.272481475877629e-12